In [1]:
from pathlib import Path

import numpy as np
import pyvista as pv

# Load Dataset

In [2]:
patient_id = "patient_01"
infile_name = Path(f"../data/raw/{patient_id}_mesh_with_uacs_fibers_tags.vtk")

In [3]:
vtk_mesh = pv.read(infile_name)
points = vtk_mesh.points
simplices = vtk_mesh.cells.reshape(-1, 4)[:, 1:]
alpha_values = vtk_mesh.point_data["alpha"]
beta_values = vtk_mesh.point_data["beta"]
fiber_directions = vtk_mesh.cell_data["fibers"]
anatomical_tags = vtk_mesh.cell_data["anatomical_tags"]

triangular_mesh = pv.PolyData.from_regular_faces(points, simplices)
triangular_mesh.point_data["alpha"] = alpha_values
triangular_mesh.point_data["beta"] = beta_values
triangular_mesh.cell_data["fibers"] = fiber_directions
triangular_mesh.cell_data["anatomical_tags"] = anatomical_tags

# Visualize UACs

In [4]:
plotter = pv.Plotter(shape=(1, 2), window_size=[1000, 600])
plotter.subplot(0, 0)
plotter.add_mesh(
    triangular_mesh.copy(),
    scalars="alpha",
    show_edges=True,
    edge_color="gray",
    edge_opacity=0.5,
    cmap="CET_R1",
    show_scalar_bar=False,
)
plotter.subplot(0, 1)
plotter.add_mesh(
    triangular_mesh.copy(),
    scalars="beta",
    show_edges=True,
    edge_color="gray",
    edge_opacity=0.5,
    cmap="CET_R1",
    show_scalar_bar=False,
)
plotter.show()

Widget(value='<iframe src="http://localhost:37161/index.html?ui=P_0x7f57fb6caa50_0&reconnect=auto" class="pyvi…

# Extract Boundaries and Adjacent Cells

In [45]:
beta_half_values = triangular_mesh.extract_values(
    scalars="beta", values=0.5, include_cells=False
)
beta_half_values_adjacent_cells = triangular_mesh.extract_values(
    scalars="beta", values=0.5
)
alpha_zero_values = triangular_mesh.extract_values(
    scalars="alpha", values=0.0, include_cells=False
)
alpha_zero_values_adjacent_cells = triangular_mesh.extract_values(
    scalars="alpha", values=0.0
)
alpha_one_values = triangular_mesh.extract_values(
    scalars="alpha", values=1.0, include_cells=False
)
alpha_one_values_adjacent_cells = triangular_mesh.extract_values(
    scalars="alpha", values=1.0
)


In [46]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    triangular_mesh,
    show_edges=True,
    scalars="anatomical_tags",
    cmap="tab20",
    show_scalar_bar=False,
    edge_color="lightgray",
    edge_opacity=0.3,
)
plotter.add_mesh(
    beta_half_values, color="firebrick", point_size=10, render_points_as_spheres=True
)
plotter.add_mesh(
    beta_half_values_adjacent_cells,
    color="firebrick",
    opacity=0.7,
    show_edges=True,
    edge_color="firebrick",
    line_width=3,
)
plotter.add_mesh(
    alpha_zero_values, color="sapgreen", point_size=10, render_points_as_spheres=True
)
plotter.add_mesh(
    alpha_zero_values_adjacent_cells,
    color="sapgreen",
    opacity=0.7,
    show_edges=True,
    edge_color="sapgreen",
    line_width=3,
)
plotter.add_mesh(
    alpha_one_values, color="cobalt", point_size=10, render_points_as_spheres=True
)
plotter.add_mesh(
    alpha_one_values_adjacent_cells,
    color="cobalt",
    opacity=0.7,
    show_edges=True,
    edge_color="cobalt",
    line_width=3,
)
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36f4d86fd0_17&reconnect=auto" class="pyv…

# Extract Actual $\beta=0.5$ Line and "Gradient Cells"

In [47]:
tolerance = 1e-6
lower_beta_cells = beta_half_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], adjacent_cells=True
)
boundary_points_beta = lower_beta_cells.extract_values(
    scalars="beta", values=0.5, include_cells=False
)

In [48]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    triangular_mesh,
    show_edges=True,
    style="wireframe",
    color="lightgray",
    edge_opacity=1,
)
plotter.add_mesh(
    triangular_mesh,
    scalars="alpha",
    opacity=1.0,
    cmap="CET_R1",
    show_scalar_bar=False,
)
plotter.add_mesh(
    lower_beta_cells,
    color="firebrick",
    opacity=1.0,
    show_edges=True,
    edge_color="firebrick",
    line_width=3,
)
plotter.add_mesh(
    boundary_points_beta,
    color="firebrick",
    point_size=10,
    render_points_as_spheres=True,
)
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36607cc050_18&reconnect=auto" class="pyv…

# Extract $\alpha = \{0,1\}$ Lines and "Gradient Cells"

In [49]:
alpha_zero_lower_beta_cells = alpha_zero_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], adjacent_cells=True
)
alpha_one_lower_beta_cells = alpha_one_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], adjacent_cells=True
)

In [50]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    triangular_mesh,
    show_edges=True,
    style="wireframe",
    color="lightgray",
    edge_opacity=1,
)
plotter.add_mesh(
    triangular_mesh,
    scalars="beta",
    opacity=1.0,
    cmap="CET_R1",
    show_scalar_bar=False,
)
plotter.add_mesh(
    alpha_zero_values, color="sapgreen", point_size=10, render_points_as_spheres=True
)
plotter.add_mesh(
    alpha_zero_lower_beta_cells,
    color="sapgreen",
    opacity=0.7,
    show_edges=True,
    edge_color="sapgreen",
    line_width=3,
)
plotter.add_mesh(
    alpha_one_values, color="cobalt", point_size=10, render_points_as_spheres=True
)
plotter.add_mesh(
    alpha_one_lower_beta_cells,
    color="cobalt",
    opacity=0.7,
    show_edges=True,
    edge_color="cobalt",
    line_width=3,
)
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36f4d84910_19&reconnect=auto" class="pyv…

# Compare in UAC Domain

In [51]:
uac_points_x = triangular_mesh.point_data["alpha"]
uac_points_y = triangular_mesh.point_data["beta"]
uac_points_z = np.zeros_like(uac_points_x)
uac_points = np.column_stack((uac_points_x, uac_points_y, uac_points_z))
uac_mesh = pv.PolyData.from_regular_faces(uac_points, simplices)
uac_mesh.cell_data["anatomical_tags"] = anatomical_tags

In [52]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    uac_mesh,
    scalars="anatomical_tags",
    show_edges=False,
    cmap="tab10",
    show_scalar_bar=False,
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36f4d85590_20&reconnect=auto" class="pyv…

In [53]:
uac_points_x_lower_beta_cells = lower_beta_cells.point_data["alpha"]
uac_points_y_lower_beta_cells = lower_beta_cells.point_data["beta"]
uac_points_z_lower_beta_cells = np.zeros_like(uac_points_x_lower_beta_cells)
uac_points_lower_beta_cells = np.column_stack(
    (
        uac_points_x_lower_beta_cells,
        uac_points_y_lower_beta_cells,
        uac_points_z_lower_beta_cells,
    )
)
uac_mesh_lower_beta_cells = pv.PolyData(uac_points_lower_beta_cells)

uac_points_x_alpha_zero_lower_beta_cells = alpha_zero_lower_beta_cells.point_data[
    "alpha"
]
uac_points_y_alpha_zero_lower_beta_cells = alpha_zero_lower_beta_cells.point_data[
    "beta"
]
uac_points_z_alpha_zero_lower_beta_cells = np.zeros_like(
    uac_points_x_alpha_zero_lower_beta_cells
)
uac_points_alpha_zero_lower_beta_cells = np.column_stack(
    (
        uac_points_x_alpha_zero_lower_beta_cells,
        uac_points_y_alpha_zero_lower_beta_cells,
        uac_points_z_alpha_zero_lower_beta_cells,
    )
)
uac_mesh_alpha_zero_lower_beta_cells = pv.PolyData(
    uac_points_alpha_zero_lower_beta_cells
)

uac_points_x_alpha_one_lower_beta_cells = alpha_one_lower_beta_cells.point_data["alpha"]
uac_points_y_alpha_one_lower_beta_cells = alpha_one_lower_beta_cells.point_data["beta"]
uac_points_z_alpha_one_lower_beta_cells = np.zeros_like(
    uac_points_x_alpha_one_lower_beta_cells
)
uac_points_alpha_one_lower_beta_cells = np.column_stack(
    (
        uac_points_x_alpha_one_lower_beta_cells,
        uac_points_y_alpha_one_lower_beta_cells,
        uac_points_z_alpha_one_lower_beta_cells,
    )
)
uac_mesh_alpha_one_lower_beta_cells = pv.PolyData(uac_points_alpha_one_lower_beta_cells)

In [54]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    uac_mesh,
    show_edges=True,
    style="wireframe",
    color="lightgray",
    edge_opacity=1,
)
plotter.add_mesh(
    uac_mesh_lower_beta_cells,
    style="points",
    color="firebrick",
    render_points_as_spheres=True,
)
plotter.add_mesh(
    uac_mesh_alpha_zero_lower_beta_cells,
    style="points",
    color="sapgreen",
    render_points_as_spheres=True,
)
plotter.add_mesh(
    uac_mesh_alpha_one_lower_beta_cells,
    style="points",
    color="cobalt",
    render_points_as_spheres=True,
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36607cd450_21&reconnect=auto" class="pyv…

# Extract Ill-Posed Sections from Global Mesh

In [55]:
beta_half_cells_to_remove = triangular_mesh.find_containing_cell(
    lower_beta_cells.cell_centers().points
)
alpha_zero_cells_to_remove = triangular_mesh.find_containing_cell(
    alpha_zero_lower_beta_cells.cell_centers().points
)
alpha_one_cells_to_remove = triangular_mesh.find_containing_cell(
    alpha_one_lower_beta_cells.cell_centers().points
)
total_cells_to_remove = np.unique(
    np.concatenate(
        (beta_half_cells_to_remove, alpha_zero_cells_to_remove, alpha_one_cells_to_remove)
    )
)
pruned_cell_inds = np.setdiff1d(np.arange(triangular_mesh.n_cells), total_cells_to_remove)
pruned_simplices = simplices[pruned_cell_inds]
pruned_mesh = pv.PolyData.from_regular_faces(points, pruned_simplices)
pruned_mesh.point_data["alpha"] = triangular_mesh.point_data["alpha"]
pruned_mesh.point_data["beta"] = triangular_mesh.point_data["beta"]

uac_points_x_pruned = pruned_mesh.point_data["alpha"]
uac_points_y_pruned = pruned_mesh.point_data["beta"]
uac_points_z_pruned = np.zeros_like(uac_points_x_pruned)
uac_points_pruned = np.column_stack((uac_points_x_pruned, uac_points_y_pruned, uac_points_z_pruned))
uac_mesh_pruned = pv.PolyData.from_regular_faces(uac_points_pruned, pruned_mesh.faces.reshape(-1, 4)[:, 1:])

In [56]:
beta_half_points = beta_half_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], include_cells=False
)
uac_points_x_beta_half = beta_half_points.point_data["alpha"]
uac_points_y_beta_half = beta_half_points.point_data["beta"]
uac_points_z_beta_half = np.zeros_like(uac_points_x_beta_half)
uac_points_beta_half = np.column_stack(
    (
        uac_points_x_beta_half,
        uac_points_y_beta_half,
        uac_points_z_beta_half,
    )
)
uac_mesh_beta_half = pv.PolyData(uac_points_beta_half)


alpha_zero_points = alpha_zero_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], include_cells=False
)
uac_points_x_alpha_zero = alpha_zero_points.point_data["alpha"]
uac_points_y_alpha_zero = alpha_zero_points.point_data["beta"]
uac_points_z_alpha_zero = np.zeros_like(uac_points_x_alpha_zero)
uac_points_alpha_zero = np.column_stack(
    (
        uac_points_x_alpha_zero,
        uac_points_y_alpha_zero,
        uac_points_z_alpha_zero,
    )
)
uac_mesh_alpha_zero = pv.PolyData(uac_points_alpha_zero)


alpha_one_points = alpha_one_values_adjacent_cells.extract_values(
    scalars="beta", ranges=[0, 0.5 - tolerance], include_cells=False
)
uac_points_x_alpha_one = alpha_one_points.point_data["alpha"]
uac_points_y_alpha_one = alpha_one_points.point_data["beta"]
uac_points_z_alpha_one = np.zeros_like(uac_points_x_alpha_one)
uac_points_alpha_one = np.column_stack(
    (
        uac_points_x_alpha_one,
        uac_points_y_alpha_one,
        uac_points_z_alpha_one,
    )
)
uac_mesh_alpha_one = pv.PolyData(uac_points_alpha_one)

In [ ]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    uac_mesh_pruned,
    show_edges=True,
    style="wireframe",
    color="lightgray",
    edge_opacity=1,
)
plotter.add_mesh(
    uac_mesh_beta_half,
    style="points",
    color="firebrick",
    opacity=1,
    render_points_as_spheres=True,
)
plotter.add_mesh(
    uac_mesh_alpha_zero,
    style="points",
    color="sapgreen",
    opacity=1,
    render_points_as_spheres=True,
)
plotter.add_mesh(
    uac_mesh_alpha_one,
    style="points",
    color="cobalt",
    opacity=1,
    render_points_as_spheres=True,
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36607cdd10_22&reconnect=auto" class="pyv…

# Distort Remaining Values to Match Boundary

In [58]:
beta_half_point_inds = []
for point in uac_mesh_beta_half.points:
    point_ind = uac_mesh_pruned.find_closest_point(point)
    beta_half_point_inds.append(point_ind)
beta_half_point_inds = np.unique(beta_half_point_inds)

alpha_zero_point_inds = []
for point in uac_mesh_alpha_zero.points:
    point_ind = uac_mesh_pruned.find_closest_point(point)
    alpha_zero_point_inds.append(point_ind)
alpha_zero_point_inds = np.unique(alpha_zero_point_inds)

alpha_one_point_inds = []
for point in uac_mesh_alpha_one.points:
    point_ind = uac_mesh_pruned.find_closest_point(point)
    alpha_one_point_inds.append(point_ind)
alpha_one_point_inds = np.unique(alpha_one_point_inds)

In [59]:
uac_mesh_pruned_distorted = uac_mesh_pruned.copy()
uac_mesh_beta_half_distorted = uac_mesh_beta_half.copy()
uac_mesh_alpha_zero_distorted = uac_mesh_alpha_zero.copy()
uac_mesh_alpha_one_distorted = uac_mesh_alpha_one.copy()

uac_mesh_pruned_distorted.points[beta_half_point_inds, 1] = 0.5
uac_mesh_pruned_distorted.points[alpha_zero_point_inds, 0] = 0.0
uac_mesh_pruned_distorted.points[alpha_one_point_inds, 0] = 1.0
uac_mesh_beta_half_distorted.points[:, 1] = 0.5
uac_mesh_alpha_zero_distorted.points[:, 0] = 0.0
uac_mesh_alpha_one_distorted.points[:, 0] = 1.0

In [60]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    uac_mesh_pruned_distorted,
    show_edges=True,
    style="wireframe",
    color="lightgray",
    edge_opacity=1,
)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:41425/index.html?ui=P_0x7f36607cead0_23&reconnect=auto" class="pyv…